<a href="https://colab.research.google.com/github/codingPhuc/coloring_puzzle_AI/blob/main/521H509.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import heapq
import numpy as np
from sympy.logic.boolalg import to_cnf
import itertools
from pysat.solvers import Glucose3

In [ ]:
 pip install python-sat==0.1.7.dev12 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00


In [100]:
def convert_to_cnf(formula):
    # Convert formula to CNF
    cnf_formula = to_cnf(formula,True ,force=True)
    return cnf_formula
def convert_to_clause(matrix):
    # initialize an empty string to hold the final result
    formula = []
    
    # loop over each row of the matrix
    for row in matrix:
        # initialize an empty list to hold the clauses for this row
        clauses = []
        
        # loop over each element of the row
        for i, element in enumerate(row):
        
            
            # generate the corresponding literal for this element
            literal = 'c_' + str(abs(element))
            
            
            
            # add a negation symbol if the element is negative
            if element < 0:
                literal = '~' + literal
            # add the literal to the list of clauses for this row
            clauses.append(literal)
        
        # join the clauses for this row with OR operators and wrap them in parentheses
        row_string = '('+  ' & '.join(clauses) + ')'
        # add the row string to the final result
        formula.append(row_string)
  
    # now it will generate 
    formula = '|'.join(formula)
    cnf_formula = convert_to_cnf(formula)
    x = str(cnf_formula).split('&')
    x = [i.replace('(', '').replace(')', '').replace('~', '-').replace('c_', '') for i in x]
    x= [i.split('|') for i in x]
    list_causes = []
    for causes in x : 
      list_causes.append(list(map(int, causes)))
    return list_causes
def generate_number_combinations(numbers, num_minus):
    """
    Generates all possible number combinations with the given number of minus signs.
    
    Args:
    numbers (list): List of numbers to generate combinations from
    num_minus (int): Number of minus signs to include in the combinations
    
    Returns:
    list: List of all possible number combinations
    """
    minus_indices = itertools.combinations(range(len(numbers)), num_minus)
    result = []
    for indices in minus_indices:
        combination = []
        for i, num in enumerate(numbers):
            if i in indices:
                combination.append(-num)
            else:
                combination.append(num)
        result.append(combination)
    return result


matrix = [[-4, -5, 7, 8], [-4, 5, -7, 8], [-4, 5, 7, -8], [4, -5, -7, 8], [4, -5, 7, -8], [4, 5, -7, -8]]
clauses = convert_to_clause(matrix)
print(clauses)


[[4, 5, 7], [4, 5, 8], [4, 7, 8], [5, 7, 8], [-4, -5, -7], [-4, -5, -8], [-4, -7, -8], [-5, -7, -8]]


In [ ]:
pip install python-sat==0.1.7.dev12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:


class Coloring_Puzzle :
    def __init__(self):
        self.M  = 0 
        self.N  =0 
        self.maze = []
        self.dictionary = dict()
    
     
    # def read_input(self, input_file):# reading the maze file 
      
    #     with open(input_file, 'r') as f:
    #         self.N, self.M = [int(it) for it in f.readline().split()]
    #         maze = [line.strip() for line in f]
    #         maze =  [it.split() for it in maze  ]
    #     return maze 
    def add_clauses(self):#function to check all the adjacnet of a point in array 
      clauses = []
      for i in range(0, self.M):
        for j in range(0 , self.N ):
          if(self.maze[i][j] !='.'):
            ad_list , len_list = self.check_adjacent((i,j))
            if(int(self.maze[i][j]) ==len_list ):
               for cl  in ad_list : 
                 clauses.append([int(cl)])
            elif(int(self.maze[i][j]) == 0): 
               
               for cl  in ad_list : 
                 clauses.append([int(-cl)])
            else: 
              num_com = generate_number_combinations(ad_list,len_list -int(self.maze[i][j]))
              list_causes = convert_to_clause(num_com)
              for l in list_causes : 
                 clauses.append(l)
              
            
      return clauses 



    def load_from_file(self, filename):# function to read file input then generate a class  array, dictionary  
      if os.path.exists(filename):
        with open(filename) as f:
          self.M, self.N= [int(it) for it in f.readline().split()]
          self.maze = [line.strip() for line in f]
          self.maze =  [it.split() for it in self.maze]
          number =1
          for i in range(0,self.M ):
            for j in range (0,self.N):
                self.dictionary[number] = (i,j) 
                number= number +1

    def find_key_by_value(self, value):
      for key, val in self.dictionary.items():
          if val == value:
              return key
      return None  # If the value is not found, return None
    def check_adjacent(self, Coordinate):
       ad_list = []
       i, j = Coordinate
       if i-1 >-1: 
         ad_list.append(self.find_key_by_value((i-1,j)))
       if i+1<  self.M: 
         ad_list.append(self.find_key_by_value((i+1,j)))
       if j-1 >-1 : 
         ad_list.append(self.find_key_by_value((i,j-1)))
       if j+1 <  self.N: 
         ad_list.append(self.find_key_by_value((i,j+1)))
       if j+1 < self.N and i-1 >-1:
         ad_list.append(self.find_key_by_value((i-1,j+1)))
       if i+1 < self.M and j-1 >-1:
         ad_list.append(self.find_key_by_value((i+1,j-1)))
       if i+1 < self.M and j+1 <self.N:
         ad_list.append(self.find_key_by_value((i+1,j+1)))
       if i-1 > -1 and j-1 > -1:
         ad_list.append(self.find_key_by_value((i-1,j-1)))
              

       ad_list.append(self.find_key_by_value((i,j)))
       return ad_list ,len(ad_list)

puzze = Coloring_Puzzle()
puzze.load_from_file('input.txt')

list_causes = puzze.add_clauses() 
s = Glucose3()
for l  in list_causes: 
   s.add_clause(l)


# Solve the SAT problem
if s.solve():
    print("SAT")
    model = s.get_model()
    print("Model:", model)
else:
    print("UNSAT")
s.delete()

SAT
Model: [-1, 2, 3, -4, -5, -6, -7, -8, 9, 10, -11, -12, -13, 14, -15, -16, -17, 18, 19, 20, -21, -22, 23, 24, 25, -26, -27, 28, 29, 30, -31, 32, 33, -34, 35, 36, -37, 38, -39, -40, -41, 42, -43, -44, -45, 46, 47, 48, 49, -50, 51, 52, -53, -54, 55, 56, -57, -58, 59, 60, 61, -62, -63, -64, 65, -66, -67, -68, -69, 70, 71, -72, -73, -74, 75, -76, -77, -78, -79, 80, 81, 82, -83, -84, 85, 86, -87, -88, 89, 90, -91, 92, 93, 94, 95, 96, 97, 98, 99, -100]


In [64]:
from pysat.solvers import Glucose3

# Create a Glucose 3 solver instance
s = Glucose3()

# Add variables to the solver


# Example usage
formula = '( c_2 &  ~c_3 & ~c_5 & ~c_6) | ( ~c_2 &  c_3 & ~c_5 & ~c_6)| ( ~c_2 &  ~c_3 & c_5 & ~c_6) | ( ~c_2 &  ~c_3 & ~c_5 & c_6)'

cnf_formula = convert_to_cnf(formula)
print(cnf_formula)
x = str(cnf_formula).split('&')
x = [i.replace('(', '').replace(')', '').replace('~', '-').replace('c_', '') for i in x]
x= [i.split('|') for i in x]

list_causes = []
for causes in x : 
   list_causes.append(list(map(int, causes)))

formula = '( ~c_4 &  ~c_5 & c_7 & c_8) | ( ~c_4 &  c_5 & ~c_7 & c_8)| ( ~c_4 &  c_5 & c_7 & ~c_8) | ( c_4 &  ~c_5 & ~c_7 & c_8)| ( c_4 &  ~c_5 & c_7 & ~c_8) | ( c_4 &  c_5 & ~c_7 & ~c_8)  '
cnf_formula = convert_to_cnf(formula)
print(cnf_formula)
x = str(cnf_formula).split('&')
x = [i.replace('(', '').replace(')', '').replace('~', '-').replace('c_', '') for i in x]
x= [i.split('|') for i in x]



for causes in x : 
   list_causes.append(list(map(int, causes)))

print(list_causes)
for l  in list_causes: 
   s.add_clause(l)

s.add_clause([-1])
s.add_clause([-2])
s.add_clause([-4])
s.add_clause([-5])


# Solve the SAT problem
if s.solve():
    print("SAT")
    model = s.get_model()
    print("Model:", model)
else:
    print("UNSAT")
s.delete()


(~c_2 | ~c_3) & (~c_2 | ~c_5) & (~c_2 | ~c_6) & (~c_3 | ~c_5) & (~c_3 | ~c_6) & (~c_5 | ~c_6) & (c_2 | c_3 | c_5 | c_6)
(c_4 | c_5 | c_7) & (c_4 | c_5 | c_8) & (c_4 | c_7 | c_8) & (c_5 | c_7 | c_8) & (~c_4 | ~c_5 | ~c_7) & (~c_4 | ~c_5 | ~c_8) & (~c_4 | ~c_7 | ~c_8) & (~c_5 | ~c_7 | ~c_8)
[[-2, -3], [-2, -5], [-2, -6], [-3, -5], [-3, -6], [-5, -6], [2, 3, 5, 6], [4, 5, 7], [4, 5, 8], [4, 7, 8], [5, 7, 8], [-4, -5, -7], [-4, -5, -8], [-4, -7, -8], [-5, -7, -8]]
SAT
Model: [-1, -2, 3, -4, -5, -6, 7, 8]


In [ ]:
def convert_to_cnf(formula):
    # Convert formula to CNF
    cnf_formula = to_cnf(formula,True)
    return cnf_formula
# Example usage
formula = '( 2 &  3 & ~ 5 & ~ 6) | ( ~2 &  3 & ~5 & ~6) '

cnf_formula = convert_to_cnf(formula)
print(cnf_formula)
x = str(cnf_formula).split('&')
x = [i.replace('(', '').replace(')', '').replace('~', '-').replace('c_', '') for i in x]
x= [i.split('|') for i in x]
s = Glucose3()
list_causes = []
for causes in x : 
   list_causes.append(list(map(int, causes)))

0
